In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces Practice

In [4]:
def pair_of_numbers(N: int, nums: list[int]) -> tuple[int, int, list[int]]:
    """
    Starting at index, check how far right you can go. Repeat same step on the leftwards direction.
    Combine the left and right to get the actual window that index i can cover (where nums[i] is the gcd and divides all)

    Problem occurs where there are duplicates where we skip those numbers since it was already covered in the window. Backward
    pass travels in the opp direction and we might end up missing the proper range.

    For eg:   2, 3, 1, 1, 3
    Forward:  0  0  3  0  0
    Backward: 0  0  0  4  0

    We could choose to ignore duplicates but that would miss nonoverlapping ranges
    We could try merging intervals but it wouldn't be trivial and we would need to merge only cases where the values match (need to lookback the entire length making it N^2)

    Simpler approach is to store the duplicates into an array and fill it in when we update i.
    """

    # Forward pass
    forward: dict[int, int] = {i: i for i in range(N)}
    i = j = 0
    match: list[int] = []
    while j <= N:
        if j < N and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            forward[i], j = j, j + 1
        else:
            for k in match:
                forward[k] = forward[i]
            i, j, match = j, j + 1, []

    # Backward pass
    backward: dict[int, int] = {i: i for i in range(N)}
    i = j = N - 1
    match = []
    while j >= -1:
        if j >= 0 and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            backward[i], j = j, j - 1
        else:
            for k in match:
                backward[k] = backward[i]
            i, j, match = j, j - 1, []

    # Merge both passes and store to a map seperated by distance
    valid_pairs: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    max_: int = 0
    for i in range(N):
        diff = forward[i] - backward[i]
        valid_pairs[diff].add(backward[i] + 1)
        max_ = max(max_, diff)

    return len(valid_pairs[max_]), max_, sorted(valid_pairs[max_])

# Testing the solution
assert pair_of_numbers(9, [2,4,3,6,9,8,10,2,4]) == (1, 3, [6])
assert pair_of_numbers(5, [4,6,9,3,6]) == (1, 3, [2])
assert pair_of_numbers(5, [1,3,5,7,9]) == (1, 4, [1])
assert pair_of_numbers(5, [2,3,5,7,11]) == (5, 0, [1,2,3,4,5])
assert pair_of_numbers(5, [1,2,3,2,1]) == (1, 4, [1])
assert pair_of_numbers(5, [2,4,1,1,2]) == (1, 4, [1])

# Good substrings - 271D
class Trie:
    def __init__(self, end: bool = False) -> None:
        self.nodes: dict[str, 'Trie'] = dict()
        self.end = end

    def __getitem__(self, ch: str) -> 'Trie':
        self.nodes[ch] = self.nodes.get(ch, Trie())
        return self.nodes[ch]

def good_substrings(S: str, good_chars_: str, K: int) -> int:
    N, count = len(S), 0
    good_chars: set[str] = {chr(ord('a') + i) for i in range(26) if good_chars_[i] == '1'}
    root: Trie = Trie()
    for i in range(N):
        k_, curr = K, root
        for j in range(i, N):
            if S[j] not in good_chars:
                k_ -= 1
            if k_ >= 0:
                curr = curr[S[j]]
                count += not curr.end
                curr.end = True
            else:
                break

    return count

# Testing the solution
assert good_substrings("ababab", "01000000000000000000000000", 1) == 5
assert good_substrings("acbacbacaa", "00000000000000000000000000", 2) == 8

LC POTD

In [5]:
# https://leetcode.com/problems/create-binary-tree-from-descriptions/submissions/1321920493/?envType=daily-question&envId=2024-07-15
def createBinaryTree(descriptions: list[list[int]]) -> BinaryTreeNode:
    """Node without parent is the root"""
    # Create the binary tree
    nodes_dict: dict[int, BinaryTreeNode] = dict()
    parents: set[int] = set()
    children: set[int] = set()
    for parent, child, direction in descriptions:
        p_node, c_node = nodes_dict.get(parent, BinaryTreeNode(parent)), nodes_dict.get(child, BinaryTreeNode(child))
        nodes_dict[parent], nodes_dict[child] = p_node, c_node

        if direction == 1:
            p_node.left = c_node
        else:
            p_node.right = c_node

        if parent not in children:
            parents.add(parent)
        if child in parents:
            parents.remove(child)
        children.add(child)

    assert len(parents) == 1
    return nodes_dict[parents.pop()]

# Testing the solution
assert createBinaryTree([[20,15,1],[20,17,0],[50,20,1],[50,80,0],[80,19,1]]).to_list() == [50,20,80,15,17,19]
assert createBinaryTree([[1,2,1],[2,3,0],[3,4,1]]).to_list() == [1,2,None,None,3,4]

KMP Algorithm

In [6]:
def create_lps_array(pattern: str) -> list[int]:
    N = len(pattern)
    pi: list[int] = [0] * N
    j = 0
    for i in range(1, N):
        while pattern[j] != pattern[i] and j > 0:
            j = pi[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        pi[i] = j

    return pi

# Testing the solution
assert create_lps_array("abcdabc") == [0, 0, 0, 0, 1, 2, 3]

Codeforces Practice

In [7]:
def longest_prefix_suffix_palindome(S: str) -> str:
    def KMP(pattern: str) -> list[int]:
        pattern_length = len(pattern)
        pi: list[int] = [0] * pattern_length
        j = 0
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = pi[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            pi[i] = j

        return pi

    N = len(S)
    i, j = 0, N - 1

    # Find greatest k such that S[0..k] + S[N - k - 1.. N - 1] is a palindrome
    k = 0
    for k in range(N // 2):
        if S[k] != S[N - k - 1]:
            break

    # Find longest prefix-suffix of S[k..N-k-1]
    middle = S[k: N - k]
    lps = middle[:KMP(middle + '#' + middle[::-1])[-1]]

    # Find longest prefix-suffix of reversed(S[k..N-k-1])
    middle_reversed = middle[::-1]
    lps_reversed = middle_reversed[:KMP(middle_reversed + '#' + middle_reversed[::-1])[-1]]

    return S[:k] + (lps if len(lps) > len(lps_reversed) else lps_reversed[::-1]) + S[N - k:]

# Testing the solution
assert len(longest_prefix_suffix_palindome("abcdfdcecba")) == len("abcdfdcba")
assert len(longest_prefix_suffix_palindome("abbaxyzyx")) == len("xyzyx")
assert len(longest_prefix_suffix_palindome("acbba")) == len("abba")
assert len(longest_prefix_suffix_palindome("a")) == len("a")
assert len(longest_prefix_suffix_palindome("codeforces")) == len("c")

#### LC Practice - Strings - KMP

In [8]:
# LC: https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/
def strStr(haystack: str, needle: str) -> int:
    def create_lps(pattern: str, M: int) -> list[int]:
        j = 0
        lps = [0] * M
        for i in range(1, M):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    N, M = len(haystack), len(needle)
    lps = create_lps(needle, M)
    j = 0
    for i in range(N):
        while haystack[i] != needle[j] and j > 0:
            j = lps[j - 1]
        if haystack[i] == needle[j]:
            j += 1
        if j >= M:
            return i - j + 1

    return -1

# Testing the solution
assert strStr("asabadzabababababaaababaabba", "abababab") == 7

In [9]:
# https://leetcode.com/problems/repeated-substring-pattern/
def repeatedSubstringPattern(S: str) -> bool:
    def create_lps(pattern: str) -> list[int]:
        pattern_length, j = len(pattern), 0
        lps: list[int] = [0] * pattern_length
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    # Create LPS, find the longest repeating pattern length
    # Ensure that the pattern repeats itself
    N = len(S)
    lps = create_lps(S)
    repeat_str_length = N - lps[-1]

    for i in range(N):
        if S[i % repeat_str_length] != S[i]:
            return False
    return (repeat_str_length < N) and (N % repeat_str_length == 0)

# Testing the solution
assert repeatedSubstringPattern("acbaaacbaa") == repeatedSubstringPattern("aaabbaaabb") == True
assert repeatedSubstringPattern("abcabcabc") == repeatedSubstringPattern("abaababaab") == True
assert repeatedSubstringPattern("abcabcabz") == repeatedSubstringPattern("aba") == False

In [10]:
# TLE :(
def watto_mechanism_trie(input_patterns: list[str], search_patterns: list[str]) -> list[bool]:
    class TrieNode:
        def __init__(self, end: bool = False):
            self.nodes: collections.defaultdict[str, 'TrieNode'] = collections.defaultdict(TrieNode)
            self.end: bool = end

        def __getitem__(self, ch: str) -> 'TrieNode':
            return self.nodes[ch]

    def search(string: str) -> bool:
        N = len(string)
        stack: list[tuple[TrieNode, int, bool]] = [(ROOT, 0, True)]
        while stack:
            curr, idx, editable = stack.pop()
            if idx == N:
                if not editable and curr.end:
                    return True
            else:
                for next_ch in ['a', 'b', 'c']:
                    if next_ch in curr.nodes:
                        if next_ch == string[idx]:
                            stack.append((curr[next_ch], idx + 1, editable))
                        elif next_ch != string[idx] and editable:
                            stack.append((curr[next_ch], idx + 1, False))
        return False

    # Convert the input patterns into a trie data strucute
    ROOT = TrieNode()
    for inp in input_patterns:
        curr = ROOT
        for ch in inp:
            curr = curr[ch]
        curr.end = True

    results: list[bool] = []
    for search_pattern in search_patterns:
        results.append(search(search_pattern))

    return results

# Testing the solution
assert watto_mechanism_trie(["aaaaa", "acacaca"], ["aabaa", "ccacacc", "caaac"]) == [True, False, False]
assert watto_mechanism_trie(["acbacbacb"], ["cbacbacb", "acbacbac", "aacbacbacb", "acbacbacbb", "acbaabacb"]) == [False, False, False, False, True]
assert watto_mechanism_trie(["ab", "cacab", "cbabc", "acc", "cacab"], ["abc", "aa", "acbca", "cb"]) == [True, True, False, True]

In [11]:
def watto_mechanism(input_patterns: list[str], search_patterns: list[str]) -> list[bool]:
    """
    Idea from editorial:
        1. Compute and store the polynomial rolling hashes for each of the input_patterns
        2. For each search string, compute possibilities and check if they exist in the hash
    """

    # Constants for polynomial rolling hash
    P, M = 257, 2 ** 61 + 7

    def poly_hash(string: str) -> int:
        nonlocal P, M
        N, hash_val, p_ = len(string), 0, 1
        for i in range(N):
            hash_val = (hash_val + (ord(string[i]) * p_)) % M
            p_ = (p_ * P) % M
        return hash_val

    def search(string: str) -> bool:
        nonlocal P, M
        N, hash_val, p_ = len(string), poly_hash(string), 1
        for i in range(N):
            for ch in ('a', 'b', 'c'):
                modified_hash = (hash_val + (ord(ch) - ord(string[i])) * p_) % M
                if ch != string[i] and modified_hash in hashes:
                    return True
            p_ = (p_ * P) % M
        return False

    # Store all the input strs as poly hashes
    hashes: set[int] = set()
    for inp in input_patterns:
        hashes.add(poly_hash(inp))

    # Iterate through the search strings, compute all hashes for all combinations in O(1) time
    results: list[bool] = []
    for inp in search_patterns:
        results.append(search(inp))

    return results

# Testing the solution
assert watto_mechanism(["aaaaa", "acacaca"], ["aabaa", "ccacacc", "caaac"]) == [True, False, False]
assert watto_mechanism(["acbacbacb"], ["cbacbacb", "acbacbac", "aacbacbacb", "acbacbacbb", "acbaabacb"]) == [False, False, False, False, True]
assert watto_mechanism(["ab", "cacab", "cbabc", "acc", "cacab"], ["abc", "aa", "acbca", "cb"]) == [True, True, False, True]

Codeforces Practice

In [12]:
def palindrome_degree(string: str) -> int:
    """
    Iterate through string, check if current substr is a palindrome.
    If it is, degree equals degree[substr // 2] + 1.
    Assume degree[substr // 2] if not found as 0.
    Return sum of all degrees.

    Checking if substr is a palindrome, check if prefix_hash hash(S[i..j]) == rev_prefix_hash hash(S[j..i])

    PREFIX_HASH = sum(ord(s[i]) * p ** i)
        abc: a + bP + cP^2

    REV_PREFIX_HASH is a little tricky but when running in a loop it is pretty simple
        abc: (rev_prefix_hash * P) + ord(s[i])
            a -> a
            b -> aP + b
            c -> aP^2 + bP + c
    """
    # Paramaters for rolling hash
    P, M = 257, 2**61 - 1

    N = len(string)
    degree: list[int] = [0 for i in range(N)]
    prefix_hash = rev_prefix_hash = ord(string[0])
    total_degree = degree[0] = 1
    p = P
    for i in range(1, N):
        prefix_hash, p = (prefix_hash + ord(string[i]) * p) % M, (P * p) % M
        rev_prefix_hash = (rev_prefix_hash * P + ord(string[i])) % M
        if prefix_hash == rev_prefix_hash:
            degree[i] = degree[(i - 1) // 2] + 1
            total_degree += degree[i]

    return total_degree

# Testing the solution
assert palindrome_degree("a2A") == 1
assert palindrome_degree("abacaba") == 6

In [13]:
def spy_syndrome2(secret: str, words: list[str]) -> str:
    # Parameters for rolling Hash
    P, M = 257, 10**9 + 7

    def poly_hash(string: str) -> int:
        nonlocal P, M
        word_length, hash_value, p = len(string), 0, 1
        for i in range(word_length):
            hash_value, p = (hash_value + p * ord(string[i].lower())) % M, (p * P) % M

        return hash_value

    # Store the list of words as hashes
    hashes: dict[int, str] = dict()
    for word in words:
        hashes[poly_hash(word)] = word

    # Compute the rev prefix hash as we iterate through the secret, if
    # match found reset and add to list of words found, note that we
    # might need to backtrack which is handled by the stack code here!
    result: list[str] = []
    secret_length = len(secret)
    stack: list[tuple[int, int]] = [(0, 0)]
    while stack:
        idx, hash_value = stack.pop()
        if idx == secret_length:
            if hash_value == 0:
                break
            else:
                result.pop()
        else:
            hash_value = (hash_value * P + ord(secret[idx].lower())) % M
            stack.append((idx + 1, hash_value))
            if hash_value in hashes:
                result.append(hashes[hash_value])
                stack.append((idx + 1, 0))

    return ' '.join(result)

# Testing the solution
assert spy_syndrome2("ariksihsidlihcdnaehsetahgnisol", ["Kira", "hates", "is", "he", "losing", "death", "childish", "L", "and", "Note"]) == "Kira is childish and he hates losing"
assert spy_syndrome2("iherehtolleh", ["HI", "Ho", "there", "HeLLo", "hello"]).lower() == "HI there HeLLo".lower()
assert spy_syndrome2("ababaaba", ["aba", "ababa"]) == "ababa aba"

Codeforces Practice

In [14]:
def xor_paths(M: int, N: int, matrix: list[list[int]], K: int) -> int:
    """
    Meet in the middle to reduce the time complexity.
    """

    valid_path_count, max_moves = 0, M + N - 2
    counts: collections.defaultdict[tuple[int, int, int], int] = collections.defaultdict(int)

    # Starting from (0, 0), make max_moves // 2: travelling right or down
    stack: list[tuple[int, int, int, int]] = [(0, 0, 0, 0)]
    while stack:
        i, j, xor_sum, moves = stack.pop()
        xor_sum ^= matrix[i][j]
        if moves == max_moves // 2:
            counts[i, j, xor_sum] += 1
        else:
            if i + 1 < M:
                stack.append((i + 1, j, xor_sum, moves + 1))
            if j + 1 < N:
                stack.append((i, j + 1, xor_sum, moves + 1))

    # Starting from (M - 1, N - 1), make max_moves - (max_moves // 2): travelling left or up
    stack = [(M - 1, N - 1, K, 0)]
    while stack:
        i, j, xor_sum, moves = stack.pop()
        if moves == max_moves - (max_moves // 2):
            # In the end counts[i, j, xor_sum] that we reached from the
            # front and the end would be such that xor_sum ^ K == 0
            valid_path_count += counts[i, j, xor_sum]
        else:
            if i - 1 >= 0:
                stack.append((i - 1, j, xor_sum ^ matrix[i][j], moves + 1))
            if j - 1 >= 0:
                stack.append((i, j - 1, xor_sum ^ matrix[i][j], moves + 1))

    return valid_path_count

# Testing the solution
assert xor_paths(3, 3, [[2, 1, 5], [7, 10, 0], [12, 6, 4]], 11) == 3
assert xor_paths(3, 4, [[1, 3, 3, 3], [0, 3, 3, 2], [3, 0, 1, 1]], 2) == 5
assert xor_paths(3, 4, [[1, 3, 3, 3], [0, 3, 3, 2], [3, 0, 1, 1]], 1000000000000000000) == 0

CP Easy: Trees

In [15]:
def subordinates(n_employees: int, bosses: list[int]) -> list[int]:
    """
    The recusive solution using cache is trivial, using the
    iterative approach to prevent stackoverflow
    """

    subordinate_counts: list[int] = [-1 for i in range(n_employees + 1)]
    subordinates: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for i, boss in enumerate(bosses, start=2):
        subordinates[boss].append(i)

    # Starting with the director, count subordinates recursively using stack
    stack: list[int] = [1]
    while stack:
        emp, subordinate_count = stack[-1], 0
        for subordinate in subordinates[emp]:
            if subordinate_counts[subordinate] == -1:
                stack.append(subordinate)
            else:
                subordinate_count += 1 + subordinate_counts[subordinate]
        if emp == stack[-1]:
            subordinate_counts[emp] = subordinate_count
            stack.pop()

    return subordinate_counts[1:]

# Testing the solution
assert subordinates(5, [1, 1, 2, 3]) == [4, 1, 1, 0, 0]
assert subordinates(10, [1,1,1,1,1,1,1,1,1]) == [9,0,0,0,0,0,0,0,0,0]
assert subordinates(10, [1,2,3,4,5,6,7,8,9]) == [9,8,7,6,5,4,3,2,1,0]

In [16]:
def tree_matching(E: int, edges: list[tuple[int, int]]) -> int:
    """
    Only having a superficial understanding, converted with trial and
    errors the solution from recursive to iterative.
    """
    adjl: collections.defaultdict[int, list] = collections.defaultdict(list)
    for i, j in edges:
        adjl[i].append(j)
        adjl[j].append(i)

    # greedily keep matching a leaf with the only vertex adjacent to it while possible
    visited: set[int] = {1}
    picked: set[int] = set()
    stack: list[tuple[int, int]] = [(1, -1)]
    matching: int = 0
    while stack:
        curr, prev = stack[-1]
        for next_ in adjl[curr]:
            if next_ not in visited:
                visited.add(next_)
                stack.append((next_, curr))
        if stack[-1] == (curr, prev):
            stack.pop()
            # At the end of each recursive call, as per recursive code logic
            # we skip the first 'edge' (1, -1)
            if prev != -1 and prev not in picked and curr not in picked:
                matching += 1
                picked.add(curr)
                picked.add(prev)

    return matching

# Testing the solution
assert tree_matching(5, [(1,2), (1,3), (3,4), (3,5)]) == 2
assert tree_matching(10, [(8,9), (10,9), (1,4), (7,1), (8,3), (10,5), (4,2), (3,7), (6,4)]) == 4

Diameter of a tree:
Naive approach is to run DFS on all nodes which would lead to a time complexity of O(n * n).<br>
Better approach would be to run DFS from any node and find the farthest point.
From this point run DFS again and fnd the farthest point.
Compute distance between these two nodes.

In [17]:
def circumference(N: int, edges: list[tuple[int, int]]) -> int:
    def farthest(node: int) -> tuple[int, int]:
        result: tuple[int, int] = (-1, -1)
        queue: collections.deque[tuple[int, int, int]] = collections.deque([(node, -1, 0)])
        while queue:
            curr, prev, dist = queue.popleft()
            if dist > result[1]:
                result = curr, dist
            for next_ in adjl[curr]:
                if next_ != prev:
                    queue.append((next_, curr, dist + 1))

        return result

    # Create an adj list from the provided edges
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Take any random node and run DFS to find the farthest point
    n1, _ = farthest(edges[0][0])
    n2, D = farthest(n1)

    # Wrong formula but this is the expected formula
    return 3 * D

# Testing the solution
assert circumference(3, [(3,2), (2,1)]) == 6
assert circumference(5, [(4,2), (1,4), (5,4), (3,4)]) == 6

LC Biweekly - 20th July 2024

In [18]:
def losingPlayer(x: int, y: int) -> str:
    alice_turn = True
    x_req, y_req = 1, 4
    while x >= x_req and y >= y_req:
        x, y = x - x_req, y - y_req
        alice_turn = not alice_turn

    return "Alice" if not alice_turn else "Bob"

# Testing the solution
assert losingPlayer(2, 7) == "Alice"
assert losingPlayer(4, 11) == "Bob"

In [19]:
def minimumLength(s: str) -> int:
    freq: collections.Counter = collections.Counter(s)
    return sum(map(lambda x: x if x <= 2 else 1 if x % 2 == 1 else 2, freq.values()))

# Testing the solution
assert minimumLength("abaacbcbb") == 5
assert minimumLength("aa") == 2

In [20]:
# TLE :)
def minChanges(nums: list[int], K: int) -> int:
    N, min_changes = len(nums), math.inf
    max_diff = max([*nums, K])
    for x in range(max_diff + 1):
        changes = 0.
        for i in range(N // 2):
            L, R = nums[i], nums[N - i - 1]
            if abs(L - R) == x:
                changes += 0
            elif (L - K <= x <= max(L, abs(L - K))) or (R - K <= x <= max(R, abs(R - K))):
                changes += 1
            elif 0 <= x <= K:
                changes += 2
            else:
                changes += math.inf

        min_changes = min(min_changes, changes)

    return int(min_changes)

# Testing the solution
assert minChanges([1,0,1,2,4,3], 4) == 2
assert minChanges([0,1,2,3,3,6,5,4], 6) == 2
assert minChanges([3,1,7,7,8,7,0,5,8,0,6,7,0,2,6,6], 8) == 6

LC Weekly Contest - 21st July 2024

In [21]:
def minBitChanges(N: int, K: int) -> int:
    changes = 0.
    while N > 0 or K > 0:
        N_bit, K_bit = N & 1, K & 1
        N, K = N >> 1, K >> 1
        if N_bit == 1 and K_bit == 0:
            changes += 1
        elif N_bit == K_bit:
            changes += 0
        else:
            changes += math.inf

    return int(changes) if not math.isinf(changes) else -1

# Testing the solution
assert minBitChanges(13, 4) == 2
assert minBitChanges(21, 21) == 0
assert minBitChanges(14, 13) == -1

In [22]:
def doesAliceWin(s: str) -> bool:
    vc = 0
    for ch in s:
        if ch in ('a', 'e', 'i', 'o', 'u'):
            vc += 1

    return vc != 0

# Testing the solution
assert doesAliceWin("leetcoder") == True
assert doesAliceWin("bcd") == False

In [23]:
def maxOperations(S: str) -> int:
    N = len(S)
    ops = ones = 0
    for i in range(N):
        if S[i] == '1':
            ones += 1
        elif S[i - 1] == '1':
            ops += ones

    return ops

# Testing the solution
assert maxOperations("1001101") == 4
assert maxOperations("00111") == 0
assert maxOperations("1000") == 1
assert maxOperations("1110") == 3

In [24]:
# Solved at 9:40 :)
def minimumOperations(nums: list[int], target: list[int]) -> int:
    def equal_sign(n1: int, n2: int) -> bool:
        return (n1 < 0 and n2 < 0) or (n1 > 0 and n2 > 0) or (n1 == n2 == 0)

    def split(left: int, right: int) -> list[tuple[int, int]]:
        result: list[tuple[int, int]] = []
        prev = left
        for i in range(left, right + 2):
            if i == right + 1 or not equal_sign(deltas[i], deltas[prev]):
                result.append((prev, i - 1))
                prev = i

        return result

    def count(left: int, right: int) -> int:
        min_ = abs(deltas[left])
        for i in range(left, right+1):
            deltas[i] = abs(deltas[i])
            min_ = min(min_, deltas[i])

        for i in range(left, right+1):
            deltas[i] -= min_

        slices: list[tuple[int, int]] = split(left, right)
        ops_ = min_
        if min_ > 0:
            for i, j in slices:
                ops_ += count(i, j)

        return ops_

    N, ops = len(nums), 0
    deltas: list[int] = [nums[i] - target[i] for i in range(N)]
    slices: list[tuple[int, int]] = split(0, N - 1)
    for i, j in slices:
        ops += count(i, j)

    return ops

# Testing the solution
assert minimumOperations([3,5,1,2], [4,6,2,4]) == 2
assert minimumOperations([1,3,2], [2,1,4]) == 5

In [ ]:
def dynamic_dyameter(N: int, edges: list[tuple[int, int]]) -> list[int]:
    def BFS(start: int) -> tuple[int, set[int]]:
        distances: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
        queue: collections.deque[tuple[int, int, int]] = collections.deque([(start, 0, 0)])
        max_dist = 0
        while queue:
            curr, prev, dist = queue.popleft()
            distances[dist].add(curr)
            max_dist = max(dist, max_dist)
            for next_ in adjl[curr]:
                if next_ != prev:
                    queue.append((next_, curr, dist + 1))

        return max_dist, distances[max_dist]

    # Convert edges to adjancency list
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n1].append(n2)
        adjl[n2].append(n1)

    # Find the farthest from a random node
    nodes = BFS(1)[1]
    leaves: set[int] = {*nodes}
    D, farthest_nodes = BFS(next(iter(leaves)))
    leaves.update(farthest_nodes)

    return [D + 1 if i in leaves else D for i in range(1, N + 1)]

# Testing the solution
assert dynamic_dyameter(3, [(3,2), (2,1)]) == [3,2,3]
assert dynamic_dyameter(5, [(4,2), (1,4), (5,4), (3,4)]) == [3,3,3,2,3]

In [ ]:
def military_problem(n_officers: int, superiors: list[int], queries: list[tuple[int, int]]) -> list[int]:
    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for officer, superior in enumerate(superiors, start=2):
        adjl[superior].append(officer)

    # DFS starting from the supreme commander - 1
    # Create variables for storing the DFS order and the start/end index for each
    order: list[int] = []
    ranges: collections.defaultdict[int, tuple[int, int]] = collections.defaultdict(lambda: (0, 0))
    stack: list[tuple[int, int]] = [(1, 0)]
    while stack:
        curr, idx = stack[-1]
        if idx == 0:
            order.append(curr)
            ranges[curr] = len(order) - 1, len(order) - 1
        if idx < len(adjl[curr]):
            stack[-1] = curr, idx + 1
            stack.append((adjl[curr][idx], 0))
        else:
            stack.pop()
            ranges[curr] = ranges[curr][0], len(order) - 1

    # Answer the queries
    result: list[int] = []
    for officer, kth in queries:
        start, end = ranges[officer]
        if end - start + 1 >= kth:
            result.append(order[start + kth - 1])
        else:
            result.append(-1)

    return result

# Testing the solution
assert military_problem(9, [1,2,2,3,5,3,6,7], [(1,5), (7,3), (1,8), (1,9)]) == [6,-1,9,4]
assert military_problem(9, [1,1,1,3,5,3,5,7], [(3,1), (1,5), (3,4), (7,3), (1,8), (1,9)]) == [3,6,8,-1,9,4]